# Fiddling around with making Bungie API calls to get json data 

## [Error Codes](https://bungie-net.github.io/multi/schema_Exceptions-PlatformErrorCodes.html#schema_Exceptions-PlatformErrorCodes)
## [Manifest](https://bungie-net.github.io/multi/operation_get_Destiny2-GetDestinyManifest.html#operation_get_Destiny2-GetDestinyManifest)


In [252]:
import numpy as np
import pandas as pd
import requests
import os, sys
import re
import json
from bs4 import BeautifulSoup

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from seekerAlexaSkill.bungie import BungieData

### Trying to scrape meta weapons from Destiny Tracker

In [128]:
page = requests.get('https://destinytracker.com/') 
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info = {"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5dfabc9f6c","applicationID":"100948194","transactionName":"ZlBSMhcEWRdWUxFQC18afTAmSn8LWlUmVgpFR18KCQBFS35eAVwc","queueTime":0,"applicationTime":6,"agent":"","atts":""}
  </script>
  <script type="text/javascript">
   window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var o=t[n]={exports:{}};e[n][0].call(o.exports,function(t){var o=e[n][1][t];return r(o||t)},o,o.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length;o++)r(n[o]);return r}({1:[function(e,t,n){function r(){}function o(e,t,n){return function(){return i(e,[f.now()].concat(u(arguments)),t?null:this,n),t?void 0:this}}var i=e("handle"),a=e(2),u=e(3),c=e("ee").get("tracer"),f=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","set

**Unfortunately, we can't get the meta list from Destiny Tracker since they're calling some asynchr method to input the info**

In [129]:
# Get PvP Meta weapons list
destinytracker.get_PvPMeta()

[<span class="name">{{ weapon.name }}</span>]


### Using the Bungie API for Destiny 2

In [253]:
cait = BungieData(api_key=os.environ["BUNGIE_API_KEY"], gamertag="BalancedSeeker6") # Never put your keys in code... export 'em!


In [254]:
# Get my own Bungie MembershipID
Me = cait.get_playerByTagName()
Me

[{'iconPath': '/img/theme/destiny/icons/icon_psn.png',
  'membershipType': 2,
  'membershipId': '4611686018453385415',
  'displayName': 'BalancedSeeker6'}]

In [255]:
# Get Destiny MembershipID
my_destiny_id = cait.get_DestinyUserId()
my_destiny_id

4611686018453385415

In [256]:
# Get my BUNGIE membershipID from using my DESTINY membershipID found in the above call
# This is the number found on the profile page of a user on Bungie.net
my_bungie_id = cait.get_BungieUserId()
my_bungie_id

11927477

### To use GetProfile, you'll need to lookup the user's Destiny membershipID by searching by gamertag. Found that in the get_DestinyUserId call : 4611686018453385415

In [257]:
My_Profile = cait.get_DestinyUserProfile(components=[100,200])
My_Profile

{'profile': {'data': {'userInfo': {'membershipType': 2,
    'membershipId': '4611686018453385415',
    'displayName': 'BalancedSeeker6'},
   'dateLastPlayed': '2018-06-21T08:00:38Z',
   'versionsOwned': 3,
   'characterIds': ['2305843009260518141',
    '2305843009289675361',
    '2305843009303434294']},
  'privacy': 1},
 'characters': {'data': {'2305843009260518141': {'membershipId': '4611686018453385415',
    'membershipType': 2,
    'characterId': '2305843009260518141',
    'dateLastPlayed': '2018-06-21T08:00:38Z',
    'minutesPlayedThisSession': '238',
    'minutesPlayedTotal': '13216',
    'light': 348,
    'stats': {'1935470627': 348,
     '2996146975': 1,
     '392767087': 10,
     '1943323491': 3},
    'raceHash': 2803282938,
    'genderHash': 2204441813,
    'classHash': 3655393761,
    'raceType': 1,
    'classType': 0,
    'genderType': 1,
    'emblemPath': '/common/destiny2_content/icons/60530e4d21f5a32bebdca82cb178c4bf.jpg',
    'emblemBackgroundPath': '/common/destiny2_con

In [258]:
# Get a random single game's stats
game_stats = cait.get_postGameStats(100)
game_stats

{'period': '2017-09-01T17:50:42Z',
 'activityDetails': {'referenceId': 1658347443,
  'directorActivityHash': 1658347443,
  'instanceId': '100',
  'mode': 2,
  'modes': [7, 2],
  'isPrivate': False},
 'entries': [{'standing': 0,
   'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
   'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/911791e90f955fc637398ea88aba74b7.jpg',
     'membershipType': 2,
     'membershipId': '4611686018468797828',
     'displayName': 'Mr_Angry'},
    'characterClass': 'Warlock',
    'classHash': 2271682572,
    'raceHash': 3887404748,
    'genderHash': 3111576190,
    'characterLevel': 1,
    'lightLevel': 100,
    'emblemHash': 1907674137},
   'characterId': '2305843009260533952',
   'values': {'assists': {'basic': {'value': 2.0, 'displayValue': '2'}},
    'completed': {'basic': {'value': 1.0, 'displayValue': 'Yes'}},
    'deaths': {'basic': {'value': 2.0, 'displayValue': '2'}},
    'kills': {'basic': {'value': 109.0, 'displayVa

In [259]:
manifest = cait.get_Manifest()
manifest

{'version': '64883.18.05.29.1341-7',
 'mobileAssetContentPath': '/common/destiny2_content/sqlite/asset/asset_sql_content_7012fd334ede7786ad089d9ceb9b20a5.content',
 'mobileGearAssetDataBases': [{'version': 0,
   'path': '/common/destiny2_content/sqlite/asset/asset_sql_content_7012fd334ede7786ad089d9ceb9b20a5.content'},
  {'version': 1,
   'path': '/common/destiny2_content/sqlite/asset/asset_sql_content_deb06e3cdc64eda41d665d489628c1b0.content'},
  {'version': 2,
   'path': '/common/destiny2_content/sqlite/asset/asset_sql_content_deb06e3cdc64eda41d665d489628c1b0.content'}],
 'mobileWorldContentPaths': {'en': '/common/destiny2_content/sqlite/en/world_sql_content_d63fda52ec4a1d85750480840e27534a.content',
  'fr': '/common/destiny2_content/sqlite/fr/world_sql_content_64bdcbcdc9907f64cb7ef7e296e415cf.content',
  'es': '/common/destiny2_content/sqlite/es/world_sql_content_8835e3094121015a7b2ca4cb8cda33aa.content',
  'de': '/common/destiny2_content/sqlite/de/world_sql_content_7dea966d758b3f8d

## Get a random statistic from Player Stats

In [260]:
# All time stats for PvE for all characters
my_stats['mergedAllCharacters']['results']['allPvE']['allTime'].keys()

dict_keys(['activitiesCleared', 'activitiesEntered', 'assists', 'totalDeathDistance', 'averageDeathDistance', 'totalKillDistance', 'kills', 'averageKillDistance', 'secondsPlayed', 'deaths', 'averageLifespan', 'bestSingleGameKills', 'bestSingleGameScore', 'opponentsDefeated', 'efficiency', 'killsDeathsRatio', 'killsDeathsAssists', 'objectivesCompleted', 'precisionKills', 'resurrectionsPerformed', 'resurrectionsReceived', 'score', 'heroicPublicEventsCompleted', 'adventuresCompleted', 'suicides', 'weaponKillsAutoRifle', 'weaponKillsFusionRifle', 'weaponKillsHandCannon', 'weaponKillsTraceRifle', 'weaponKillsPulseRifle', 'weaponKillsRocketLauncher', 'weaponKillsScoutRifle', 'weaponKillsShotgun', 'weaponKillsSniper', 'weaponKillsSubmachinegun', 'weaponKillsRelic', 'weaponKillsSideArm', 'weaponKillsSword', 'weaponKillsAbility', 'weaponKillsGrenade', 'weaponKillsGrenadeLauncher', 'weaponKillsSuper', 'weaponKillsMelee', 'weaponBestType', 'allParticipantsCount', 'allParticipantsScore', 'allParti

In [261]:
# All time stats for PvP for all characters
my_stats['mergedAllCharacters']['results']['allPvP']['allTime'].keys()

dict_keys(['activitiesEntered', 'activitiesWon', 'assists', 'totalDeathDistance', 'averageDeathDistance', 'totalKillDistance', 'kills', 'averageKillDistance', 'secondsPlayed', 'deaths', 'averageLifespan', 'score', 'averageScorePerKill', 'averageScorePerLife', 'bestSingleGameKills', 'bestSingleGameScore', 'opponentsDefeated', 'efficiency', 'killsDeathsRatio', 'killsDeathsAssists', 'objectivesCompleted', 'precisionKills', 'resurrectionsPerformed', 'resurrectionsReceived', 'suicides', 'weaponKillsAutoRifle', 'weaponKillsFusionRifle', 'weaponKillsHandCannon', 'weaponKillsTraceRifle', 'weaponKillsPulseRifle', 'weaponKillsRocketLauncher', 'weaponKillsScoutRifle', 'weaponKillsShotgun', 'weaponKillsSniper', 'weaponKillsSubmachinegun', 'weaponKillsRelic', 'weaponKillsSideArm', 'weaponKillsSword', 'weaponKillsAbility', 'weaponKillsGrenade', 'weaponKillsGrenadeLauncher', 'weaponKillsSuper', 'weaponKillsMelee', 'weaponBestType', 'winLossRatio', 'allParticipantsCount', 'allParticipantsScore', 'allP

In [262]:
play_type = np.random.choice(['allPvP', 'allPvE'])
play_stats = my_stats['mergedAllCharacters']['results'][play_type]['allTime']
print("Play Type: {}\n{}".format(play_type, play_stats))

Play Type: allPvP
{'activitiesEntered': {'statId': 'activitiesEntered', 'basic': {'value': 393.0, 'displayValue': '393'}}, 'activitiesWon': {'statId': 'activitiesWon', 'basic': {'value': 172.0, 'displayValue': '172'}}, 'assists': {'statId': 'assists', 'basic': {'value': 1158.0, 'displayValue': '1158'}, 'pga': {'value': 2.946564885496183, 'displayValue': '2.9'}}, 'totalDeathDistance': {'statId': 'totalDeathDistance', 'basic': {'value': 0.0, 'displayValue': '0.0'}}, 'averageDeathDistance': {'statId': 'averageDeathDistance', 'basic': {'value': 0.0, 'displayValue': '0.0'}}, 'totalKillDistance': {'statId': 'totalKillDistance', 'basic': {'value': 56860.0, 'displayValue': '56860.0'}}, 'kills': {'statId': 'kills', 'basic': {'value': 3751.0, 'displayValue': '3751'}, 'pga': {'value': 9.544529262086513, 'displayValue': '9.5'}}, 'averageKillDistance': {'statId': 'averageKillDistance', 'basic': {'value': 15.158624366835511, 'displayValue': '15.2'}}, 'secondsPlayed': {'statId': 'secondsPlayed', 'bas

In [263]:
rand_stat_key = np.random.choice(list(play_stats.keys()))
rand_stat_key

'averageScorePerLife'

In [264]:
' '.join(re.findall('[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', rand_stat_key))

'average Score Per Life'

In [265]:
random_stat = play_stats[rand_stat_key]
random_stat

{'statId': 'averageScorePerLife',
 'basic': {'value': 1.8127174944637774, 'displayValue': '1.81'}}

In [266]:
random_stat['basic']['displayValue']

'1.81'

### Stat Definitions

In [267]:
stat_definitions = cait.get_StatDefinitions()
stat_definitions

{'activitiesCleared': {'statId': 'activitiesCleared',
  'group': 1,
  'periodTypes': [2, 1],
  'modes': [7, 6, 4, 2, 3, 16, 17, 46, 47, 18, 58],
  'category': 0,
  'statName': 'Activities Cleared',
  'statDescription': 'Successful completions of this mission type.',
  'unitType': 1,
  'unitLabel': '',
  'weight': 1},
 'activitiesEntered': {'statId': 'activitiesEntered',
  'group': 1,
  'periodTypes': [2, 1],
  'modes': [0,
   7,
   6,
   5,
   4,
   2,
   3,
   16,
   17,
   46,
   47,
   18,
   19,
   10,
   12,
   37,
   38,
   31,
   45,
   44,
   43,
   39,
   41,
   42,
   40,
   49,
   50,
   15,
   25,
   48,
   32,
   51,
   52,
   54,
   53,
   55,
   56,
   57,
   58,
   59],
  'category': 8,
  'statName': 'Activities Entered',
  'statDescription': 'Battles fought - some lost, some won.',
  'unitType': 1,
  'unitLabel': '',
  'weight': 1},
 'activitiesWon': {'statId': 'activitiesWon',
  'group': 1,
  'periodTypes': [2, 1],
  'modes': [5,
   19,
   10,
   12,
   37,
   38,
   

### Get Post Game Carnage Reports
>"If you want it specifically for you, you can use the *GetActivityHistory* endpoint to get all of your PGCR IDs (grab the **referenceId** property on the **DestinyHistoricalStatsActivity** objects being returned), and then use that to make calls to the *GetPostGameCarnageReport* endpoint!" - vthornheart-bng https://github.com/Bungie-net/api/issues/452


> "In the **PGCR** endpoint, there is a **referenceId**, which is the **activityHash** in **AggregateActivityStats** 
To get a player's match history, you need to use the GetActivityHistory endpoint  
GetActivityHistory = List of their recent matches  
GetPostGameCarnageReport = A specific match  
GetDestinyAggregateActivityStats = Aggregation of those matches per activityHash  - https://discordapp.com/channels/296008008956248066/296008136785920001/403851740686057472"

In [268]:
# Need a list of character IDs
My_Profile['profile']['data']['characterIds']

['2305843009260518141', '2305843009289675361', '2305843009303434294']

In [269]:
# Get activities for my Titan main
site_call = "https://bungie.net/Platform/Destiny2/2/Account/" + str(4611686018453385415)+\
    "/Character/" + str(2305843009260518141) + "/Stats/Activities/?count=250" 
request = requests.get(site_call, 
                       headers={"X-API-Key":os.environ["BUNGIE_API_KEY"]})
request.json()['Response']

{'activities': [{'period': '2018-06-21T07:38:59Z',
   'activityDetails': {'referenceId': 1489679220,
    'directorActivityHash': 2491884566,
    'instanceId': '1827574202',
    'mode': 57,
    'modes': [5, 32, 57],
    'isPrivate': True},
   'values': {'assists': {'statId': 'assists',
     'basic': {'value': 4.0, 'displayValue': '4'}},
    'score': {'statId': 'score',
     'basic': {'value': 16.0, 'displayValue': '16'}},
    'kills': {'statId': 'kills',
     'basic': {'value': 16.0, 'displayValue': '16'}},
    'averageScorePerKill': {'statId': 'averageScorePerKill',
     'basic': {'value': 1.0, 'displayValue': '1.00'}},
    'deaths': {'statId': 'deaths',
     'basic': {'value': 34.0, 'displayValue': '34'}},
    'averageScorePerLife': {'statId': 'averageScorePerLife',
     'basic': {'value': 0.45714285714285713, 'displayValue': '0.46'}},
    'completed': {'statId': 'completed',
     'basic': {'value': 1.0, 'displayValue': 'Yes'}},
    'opponentsDefeated': {'statId': 'opponentsDefeated',

In [270]:
from collections import defaultdict

character_activities = {}

# Bungie API maxes out the count returned at 250 for whatever reason
for character in My_Profile['profile']['data']['characterIds']:
    activity_ids = [act['activityDetails']['referenceId'] for act in cait.get_ActivityHistory(character, count=50)['activities']]
    character_activities[character] = activity_ids

In [271]:
len(character_activities['2305843009260518141'])

50

In [273]:
character_activities

{'2305843009260518141': [1489679220,
  3897312654,
  2693136600,
  2693136600,
  2693136600,
  2046332536,
  806094750,
  2748633318,
  3143798436,
  4166562681,
  3897312654,
  1702649201,
  451430877,
  1233767907,
  1233767907,
  2262757213,
  4012915511,
  1233767907,
  3233852802,
  2271820498,
  3897312654,
  451430877,
  1435054848,
  1435054848,
  3897312654,
  3233852802,
  806094750,
  2473919228,
  4012915511,
  2666761222,
  2473919228,
  2262757213,
  750001803,
  3164915257,
  2473919228,
  1003889713,
  148937731,
  990984849,
  1489679220,
  3089205900,
  2693136603,
  2693136603,
  451430877,
  958578340,
  2724706103,
  649648599,
  649648599,
  3676029623,
  3734723183,
  3349246768],
 '2305843009289675361': [1512980468,
  1057017675,
  399506119,
  750001803,
  2666761222,
  778271008,
  3143798436,
  778271008,
  806094750,
  399506119,
  1711620427,
  1711620427,
  3849796864,
  2666761222,
  1711620427,
  806094750,
  778271008,
  399506119,
  3164915257,
  31437

In [274]:
site_call = "https://bungie.net/Platform/Destiny2/Stats/PostGameCarnageReport/" + str(1489679220)
request = requests.get(site_call, 
                        headers={"X-API-Key":os.environ["BUNGIE_API_KEY"]})
request.json()['Response']

{'period': '2018-02-16T22:11:10Z',
 'activityDetails': {'referenceId': 1563393783,
  'directorActivityHash': 1984315274,
  'instanceId': '1489679220',
  'mode': 3,
  'modes': [7, 3, 18],
  'isPrivate': False},
 'entries': [{'standing': 0,
   'score': {'basic': {'value': 0.0, 'displayValue': '0'}},
   'player': {'destinyUserInfo': {'iconPath': '/common/destiny2_content/icons/86724d21bf854a3824d0f60b01594060.jpg',
     'membershipType': 4,
     'membershipId': '4611686018471726120',
     'displayName': 'vimpelkid'},
    'characterClass': 'Titan',
    'classHash': 3655393761,
    'raceHash': 898834093,
    'genderHash': 3111576190,
    'characterLevel': 25,
    'lightLevel': 318,
    'emblemHash': 1724152705},
   'characterId': '2305843009310665352',
   'values': {'assists': {'basic': {'value': 27.0, 'displayValue': '27'}},
    'completed': {'basic': {'value': 1.0, 'displayValue': 'Yes'}},
    'deaths': {'basic': {'value': 6.0, 'displayValue': '6'}},
    'kills': {'basic': {'value': 87.0,

In [272]:
# Now that we have our referenceIds, pull the PGCRs for each activity
PGCR = {}
for character in character_activities:
    for activity in character:
        PGCR[activity] = cait.get_PGCR(activity)

PGCR

KeyError: 'Response'